In [2]:
from collections import defaultdict
from count_syllables import count_syllables
from nltk.corpus import cmudict
from spacy.lang.en import English
from string import punctuation
import json
import logging
import nltk
import os
import pandas as pd
import pprint
import random
import re
import shutil
from tqdm import tqdm
import spacy
import sys
import tracery

In [3]:
logging.disable(
    logging.CRITICAL
)  # comment out to enable debugging messages logging.basicConfig(level=logging.DEBUG, format='%(message)s')

In [4]:
nltk.download("cmudict")

[nltk_data] Downloading package cmudict to /Users/bean/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [33]:
with open("../haikuifier/lesson.txt") as f:
    lesson = f.read()
    lesson_list = list(lesson.replace(" ", ", "))
lesson_list1

/var/folders/m9/kw6252cj3lqgftmsg97nw4z80000gn/T/ipykernel_17985/2263952448.py:4: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  lesson_list1 = random.shuffle(lesson_list, random)


TypeError: 'module' object is not callable

In [32]:
lesson_list

'e'

In [19]:
lesson = random.shuffle(lesson)

TypeError: 'str' object does not support item assignment

In [47]:
haikucorp = lesson
haikuraw = haikucorp.str.replace("/", "")
haikuraw = haikucorp.str.replace("/", "")
haikuraw = haikuraw.str.replace("  ", " ")
haikuraw = haikuraw.str.replace(".", "")
rawlist = haikuraw.to_list()
string = " ".join(rawlist)
with open("haikucorpus.txt", "w") as outfile:
    outfile.write(string)

/var/folders/m9/kw6252cj3lqgftmsg97nw4z80000gn/T/ipykernel_63382/169025837.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  haikuraw = haikuraw.str.replace(".", "")


In [10]:
filenames = ["train.txt", "lesson.txt"]
with open("trainer.txt", "w") as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())

In [11]:
def load_training_file(file):
    """Return text file as string."""
    with open(file) as f:
        raw_haiku = f.read()
        return raw_haiku


def prep_training(raw_haiku):
    """Load string, remove newline, split words on spaces, and return list."""
    corpus = raw_haiku.replace("\n", " ").split()
    return corpus

In [12]:
def map_word_to_word(corpus):
    """Load list & use dictionary to map word to word that follows"""
    limit = len(corpus) - 1
    dict1_to_1 = defaultdict(list)
    for index, word in enumerate(corpus):
        if index < limit:
            suffix = corpus[index + 1]
            dict1_to_1[word].append(suffix)
    logging.debug('map_word_to_word results for "sake" = %s\n', dict1_to_1["sake"])
    return dict1_to_1


def map_2_words_to_word(corpus):
    """Load list and use dictionary to map word-pair to trailing word."""
    limit = len(corpus) - 2
    dict2_2_1 = defaultdict(list)
    for index, word in enumerate(corpus):
        if index < limit:
            key = word + " " + corpus[index + 1]
            suffix = corpus[index + 2]
            dict2_2_1[key].append(suffix)
    logging.debug(
        'map_2_words_to_word results for "sake jug" = %s\n', dict2_2_1["sake jug"]
    )
    return dict2_2_1

In [13]:
def random_word(corpus):
    """Return random word and syllable count from training corpus."""
    word = random.choice(corpus)
    num_syls = count_syllables(word)
    if num_syls > 4:
        random_word(corpus)
    else:
        logging.debug("random word and syllables = %s %s\n", word, num_syls)
        return (word, num_syls)

In [14]:
def word_after_single(prefix, suffix_map_1, current_syls, target_syls):
    """Return all acceptable words in a corpus that follow a single word"""
    accepted_words = []
    suffixes = suffix_map_1.get(prefix)
    if suffixes != None:
        for candidate in suffixes:
            num_syls = count_syllables(candidate)
            if current_syls + num_syls <= target_syls:
                accepted_words.append(candidate)
    logging.debug('accepted words after "%s" = %s\n', prefix, set(accepted_words))
    return accepted_words

In [15]:
def word_after_double(prefix, suffix_map_2, current_syls, target_syls):

    """Return all acceptable words in a corpus that follow a word pair."""

    accepted_words = []
    suffixes = suffix_map_2.get(prefix)
    if suffixes != None:
        for candidate in suffixes:
            num_syls = count_syllables(candidate)
            if current_syls + num_syls <= target_syls:
                accepted_words.append(candidate)
    logging.debug('accepted words after "%s" = %s\n', prefix, set(accepted_words))
    return accepted_words

In [16]:
def haiku_line(suffix_map_1, suffix_map_2, corpus, end_prev_line, target_syls):
    """Build a haiku line from a training corpus and return it."""
    line = "2/3"
    line_syls = 0
    current_line = []
    if len(end_prev_line) == 0:  # first line
        line = "1"
        word, num_syls = random_word(corpus)
        current_line.append(word)
        line_syls += num_syls
        word_choices = word_after_single(word, suffix_map_1, line_syls, target_syls)

        while len(word_choices) == 0:
            prefix = random.choice(corpus)
            logging.debug("new random prefix = %s %s", word, num_syls)
            word_choices = word_after_single(
                prefix, suffix_map_1, line_syls, target_syls
            )
        word = random.choice(word_choices)
        num_syls = count_syllables(word)
        logging.debug("word and syllables = %s %s", word, num_syls)
        line_syls += num_syls
        current_line.append(word)
        if line_syls == target_syls:
            end_prev_line.extend(current_line[-2:])
            return current_line, end_prev_line
    else:  # lines 2 + 3
        current_line.extend(end_prev_line)

    while True:
        logging.debug("line = %s\n", line)
        prefix = current_line[-2] + " " + current_line[-1]
        word_choices = word_after_double(prefix, suffix_map_2, line_syls, target_syls)

        while len(word_choices) == 0:
            prefix = random.choice(corpus)
            logging.debug("new random prefix = %s %s", word, num_syls)
            word_choices = word_after_double(
                prefix, suffix_map_2, line_syls, target_syls
            )

        word = random.choice(word_choices)
        num_syls = count_syllables(word)
        logging.debug("word and syllables = %s %s", word, num_syls)

        if line_syls + num_syls > target_syls:
            continue
        elif line_syls + num_syls < target_syls:
            current_line.append(word)
            line_syls += num_syls
        elif line_syls + num_syls == target_syls:
            current_line.append(word)
            break

    end_prev_line = []
    end_prev_line.extend(current_line[-2:])

    if line == "1":
        final_line = current_line[:]
    else:
        final_line = current_line[2:]

    return final_line, end_prev_line

In [17]:
def main():
    """Give user choice of building a haiku or modifying an existing haiku"""
    intro = """\n
    A N-monkeys at N-typewriters for N-millenia...
    or one computer piped full of garbage...
    can sometimes produce haiku.\n"""
    print("{}".format(intro))
    
    raw_haiku = load_training_file("trainer.txt")
    corpus = prep_training(raw_haiku)
    suffix_map_1 = map_word_to_word(corpus)
    suffix_map_2 = map_2_words_to_word(corpus)
    final = []
    
    choice = None
    while choice != "0":
        
        print(
            """
            
            TMZ Haiku Generator
            
            0 - QUIT — I'm done having fun.
            1 - Generate a haiku
            2 - Regenerate Line 2
            3 - Regenerate Line 3
            
            """
        )
        
        choice = input("Choice: ")
        print()
        
        # exit
        if choice == "0":
            print("Coward.")
            sys.exit()
            
            # generate full haiku
        elif choice == tqdm("1"):
            final = []
            end_prev_line = []
            first_line, end_prev_line1 = haiku_line(suffix_map_1, suffix_map_2, corpus, end_prev_line, 5)
            final.append(first_line)
            line, end_prev_line2 = haiku_line(suffix_map_1, suffix_map_2, corpus, end_prev_line1, 7)
            final.append(line)
            line, end_prev_line3 = haiku_line(suffix_map_1, suffix_map_2, corpus, end_prev_line2, 5)
            final.append(line)
            
        elif choice == "2":
            if not final:
                print("You've gotta generate a haiku first, silly. Try Option 1.")
                continue
            else:
                line, end_prev_line2 = haiku_line(suffix_map_1, suffix_map_2, corpus, end_prev_line1, 7)
                final[1] = line
                
            # Regenerate line 3
        elif choice == "3":
            if not final:
                print("I can't regenerate nothing, doofus. Try Option 1.")
                continue
            else:
                line, end_prev_line2 = haiku_line(suffix_map_1, suffix_map_2, corpus, end_prev_line_2, 5)
                final[2] = line
                
        # Invalid choice
        else:
            print("\nYou need to pretend that your keyboard only has five keys: 0 1 2 3 and RETURN. Try pressing a cardinal number and then RETURN on your five-keyed keyboard. Whatever you just pushed isn't a choice I can understand.", file=sys.stderr)
            continue
        
        #display results
        print(print("First line = ", end=""))
        print(' '.join(final[0]), file=sys.stderr)
        print("Second line = ", end="")
        print(" ".join(final[1]), file=sys.stderr)
        print("Third line = ", end="")
        print(" ".join(final[2]), file=sys.stderr)
        print()
        
    input("\n\nPress the RETURN key to exit, friend.")
    
if __name__ == '__main__':
    main()



    A N-monkeys at N-typewriters for N-millenia...
    or one computer piped full of garbage...
    can sometimes produce haiku.


            
            TMZ Haiku Generator
            
            0 - QUIT — I'm done having fun.
            1 - Generate a haiku
            2 - Regenerate Line 2
            3 - Regenerate Line 3
            
            



  0%|          | 0/1 [00:00<?, ?it/s]

AttributeError: 'str' object has no attribute '_comparable'

In [3]:

nlp = spacy.load('en_core_web_md')
nlp_en = English()

In [4]:
headlines = pd.read_csv('../headlines.csv')
headline_list = headlines.headline.to_list()
headline_str = str(headline_list)
doc = nlp(headline_str)

In [5]:
sentences = list(doc.sents)
words = [w for w in list(doc) if w.is_alpha]
noun_chunks = list(doc.noun_chunks)
entities = list(doc.ents)
nouns = [w for w in words if w.pos_ == "NOUN"]
verbs = [w for w in words if w.pos_ == "VERB"]
adjs = [w for w in words if w.pos_ == "ADJ"]
advs = [w for w in words if w.pos_ == "ADV"]

In [6]:
print("len of —")
print(f"sentences: {len(sentences)}")
print(f"words: {len(words)}")
print(f"noun chunks: {len(noun_chunks)}")
print(f"entities: {len(entities)}")
print("- - -")
print(f"nouns: {len(nouns)}")
print(f"verbs: {len(verbs)}")
print(f"adjs: {len(adjs)}")
print(f"advs: {len(advs)}")


len of —
sentences: 121
words: 1255
noun chunks: 379
entities: 185
- - -
nouns: 91
verbs: 128
adjs: 17
advs: 30


In [7]:
import random
for item in random.sample(noun_chunks, 5):
    print(item.text.strip().replace("\n", " "))
    print()

In-Game Brawl

IG Model Gena Tew AIDS

", ' Celebrity Scramble Guess Who! ', ' Jake Paul Hasim Rahman Jr.

Hours

', ' Ukraine Prez Volodymyr Zelensky Strong Support



In [8]:
sentence_strs = [item.text for item in doc.sents]

In [9]:
random.sample(sentence_strs, 10)

['\'I Think It\'s Going To Get Done\' ", \' July 2022 Hot Shots Bae Watch Alert! \', " HEROIC PIZZA DELIVERY',
 '", \' Britney Spears Will Not Be Deposed, Judge Rules \', \' Jeff Bezos Parents Buy $34M Mega Mansion in FL \', " Joey Chestnut Downs 44 Cane\'s Chicken Fingers In Five Minutes!!! ...',
 "For Bashing Decision overturning Roe v. Wade ', ' Bob Dylan Accuser Drops Sex Abuse Suit ', ' Jeff Ross Jon Stewart Feels the Pain of Veterans ...",
 'Bare at DT\'s Golf Course ", \' China Rocket Debris Expected To Rain Down On Earth ... AGAIN!!!',
 '\', \' Taco Bell Employee Allegedly Throws Boiling Hot Water at Customers\', " Brittney Griner Sends Message To Wife In Court ... \'Good Luck',
 'All These People in Red!!!',
 '\', " Britney Spears My Book is Done!!! ...',
 '", \' Luci Fang Sink Your Teeth Into Her Hot Shots!',
 'Former Cops Get Prison Time ...',
 'Wifey Likes to Party Too!!!']

In [10]:
for item in random.sample(nouns, 5): # change "nouns" to "verbs" or "adjs" or "advs" to sample from those lists!
    print(item.text)

Dancers
Pain
Biggie
Crowd
Drago


In [11]:
people = [e for e in entities if e.label_ == "PERSON"]
locations = [e for e in entities if e.label_ == "LOC"]
times = [e for e in entities if e.label_ == "TIME"]

In [12]:
from collections import Counter
word_count = Counter([w.text for w in words])

In [14]:
word_count['Britney']

3

In [15]:
word_count.most_common(10)

[('I', 17),
 ('to', 14),
 ('For', 13),
 ('To', 12),
 ('at', 11),
 ('My', 10),
 ('Star', 9),
 ('for', 9),
 ('of', 7),
 ('In', 7)]

In [16]:
with open("words.txt", "w") as fh:
    fh.write("\n".join([w.text for w in words]))
shutil.move("words.txt", "data")

'data'

In [17]:
# saves a list of spacy values to a text file in ../data
def save_spacy_list(filename, t):
    with open(filename, "w") as fh:
        fh.write("\n".join([item.text for item in t]))
        ## If file exists, delete it ##
    path = "f'..data/{filename}'"
    if os.path.isfile(path):
        os.remove(path)
    else:## Show an error ##
        print(f"Error: {path} file not found")
    shutil.move(filename, "../data")

In [18]:
spacy_values = [
    "noun_chunks",
    "entities",
    "words",
    "adjs",
    "advs",
    "verbs",
    "nouns"
]

In [20]:
save_spacy_list("words.txt", words)
save_spacy_list("noun_chunks.txt", noun_chunks)
save_spacy_list("entities.txt", entities)
save_spacy_list("adjs.txt", adjs)
save_spacy_list("advs.txt", advs)
save_spacy_list("verbs.txt", verbs)
save_spacy_list("nouns.txt", nouns)

Error: f'..data/{filename}' file not found
Error: f'..data/{filename}' file not found
Error: f'..data/{filename}' file not found
Error: f'..data/{filename}' file not found
Error: f'..data/{filename}' file not found
Error: f'..data/{filename}' file not found
Error: f'..data/{filename}' file not found


In [21]:
def save_counter_tsv(filename, counter, limit=1000):
    with open(filename, "w") as outfile:
        outfile.write("key\tvalue\n")
        for item, count in counter.most_common():
            outfile.write(item.strip() + "\t" + str(count) + "\n")
    shutil.move(filename, "../data")

In [23]:
save_counter_tsv("100_common_words.tsv", word_count, 100)

In [25]:
people_counter = Counter([e.text.lower() for e in people])
save_counter_tsv("people_count.tsv", people_counter, 100)

In [26]:
for word in random.sample(words, 12):
    print(word.text, "→", word.lemma_)

During → during
Tryout → Tryout
Prison → Prison
Fraud → Fraud
Disrespectful → disrespectful
Savarino → Savarino
Being → be
The → the
at → at
Too → Too
Bday → Bday
Beyonce → Beyonce


In [27]:
sentence = random.choice(sentences)
for word in sentence:
    print(word.text)

Ideas
to
End


In [28]:
for item in random.sample(words, 24):
    print(item.text, "/", item.pos_, "/", item.tag_)

MORE / ADJ / JJR
Hobby / PROPN / NNP
WWE / PROPN / NNP
Weight / PROPN / NNP
Single / PROPN / NNP
Her / PRON / PRP$
Selfie / PROPN / NNP
a / DET / DT
Ukrainian / PROPN / NNP
Sample / PROPN / NNP
Days / NOUN / NNS
Ric / PROPN / NNP
You / PRON / PRP
Too / ADV / RB
Market / NOUN / NN
In / ADP / IN
To / ADP / IN
Frigin / PROPN / NNP
Old / PROPN / NNP
It / PRON / PRP
Shows / VERB / VBZ
Print / VERB / VB
Out / NOUN / NN
a / DET / DT


In [29]:
spacy.explain('SCONJ')

'subordinating conjunction'

In [30]:
only_past = [item.text for item in doc if item.tag_ == 'VBN']

In [31]:
random.sample(only_past, 2)

['Canceled', 'Killed']

In [32]:
only_plural = [item.text for item in doc if item.tag_ == 'NNS']

In [33]:
sent = random.choice(sentences)
print("Original sentence:", sent.text.replace("\n", " "))
for word in sent:
    print()
    print("Word:", word.text)
    print("Tag:", word.tag_)
    print("Head:", word.head.text)
    print("Dependency relation:", word.dep_)
    print("Children:", list(word.children))

Original sentence: As Monkeypox Cases Keep Stacking Up ', ' Mega Millions Lottery Single Winner Snags $1.28 BILLION!!!

Word: As
Tag: IN
Head: Keep
Dependency relation: mark
Children: []

Word: Monkeypox
Tag: NNP
Head: Cases
Dependency relation: compound
Children: []

Word: Cases
Tag: NNS
Head: Keep
Dependency relation: nsubj
Children: [Monkeypox]

Word: Keep
Tag: VBP
Head: Keep
Dependency relation: ROOT
Children: [As, Cases, Stacking, ', ,, Snags, !, !, !]

Word: Stacking
Tag: VBG
Head: Keep
Dependency relation: xcomp
Children: [Up]

Word: Up
Tag: RP
Head: Stacking
Dependency relation: prt
Children: []

Word: '
Tag: ''
Head: Keep
Dependency relation: punct
Children: []

Word: ,
Tag: ,
Head: Keep
Dependency relation: punct
Children: []

Word: '
Tag: ''
Head: Snags
Dependency relation: punct
Children: []

Word: Mega
Tag: NNP
Head: Millions
Dependency relation: compound
Children: []

Word: Millions
Tag: NNPS
Head: Lottery
Dependency relation: compound
Children: [Mega]

Word: Lottery
Tag:

- nsubj: this word's head is a verb, and this word is itself the subject of the verb
- nsubjpass: same as above, but for subjects in sentences in the passive voice
- dobj: this word's head is a verb, and this word is itself the direct object of the verb
- iobj: same as above, but indirect object
- aux: this word's head is a verb, and this word is an "auxiliary" verb (like "have", "will", "be")
- attr: this word's head is a copula (like "to be"), and this is the description attributed to the subject of the sentence (e.g., in "This product is a global brand", brand is dependent on is with the attr dependency relation)
- det: this word's head is a noun, and this word is a determiner of that noun (like "the," "this," etc.)
- amod: this word's head is a noun, and this word is an adjective describing that noun
- prep: this word is a preposition that modifies its head
- pobj: this word is a dependent (object) of a preposition

In [34]:
def flatten_subtree(st):
    return ''.join([w.text_with_ws for w in list(st)]).strip()

In [35]:
sent = random.choice(sentences)
print("Original sentence:", sent.text.replace("\n", " "))
for word in sent:
    print()
    print("Word:", word.text.replace("\n", " "))
    print("Flattened subtree: ", flatten_subtree(word.subtree).replace("\n", " "))

Original sentence: But I'm OK!!!

Word: But
Flattened subtree:  But

Word: I
Flattened subtree:  I

Word: 'm
Flattened subtree:  But I'm OK!!!

Word: OK
Flattened subtree:  OK

Word: !
Flattened subtree:  !

Word: !
Flattened subtree:  !

Word: !
Flattened subtree:  !


In [36]:
subjects = [flatten_subtree(word.subtree) for word in doc if word.dep_ in ('nsubj', 'nsubjpass')]

In [37]:
random.sample(subjects, 12)

['Dave Chappelle',
 'We',
 'Sen. Toomey',
 'AP Bari New Maybach',
 'This Georgia Peach',
 "Jeff Bezos' Parents",
 "Beyonce 'Renaissance'",
 'I',
 'What',
 'Russia',
 'Fight',
 'I']

In [38]:
prep_phrases = [flatten_subtree(word.subtree).replace("\n", " ") for word in doc if word.dep_ == 'prep']

In [39]:
subjects = [flatten_subtree(word.subtree).replace("\n", " ")
            for word in doc if word.dep_ in ('nsubj', 'nsubjpass')]
past_tense_verbs = [word.text for word in words if word.tag_ == 'VBD' and word.lemma_ != 'be']
adjectives = [word.text for word in words if word.tag_.startswith('JJ')]
nouns = [word.text for word in words if word.tag_.startswith('NN')]
prep_phrases = [flatten_subtree(word.subtree).replace("\n", " ")
                for word in doc if word.dep_ == 'prep']

In [40]:
import tracery
from tracery.modifiers import base_english

In [46]:
rules = {
    "origin": [
        "#subject.capitalize# #predicate#.",
        "#subject.capitalize# #predicate#.",
        "#prepphrase.capitalize#, #subject# #predicate#."
    ],
    "predicate": [
        "#verb#",
        "#verb# #nounphrase#",
        "#verb# #prepphrase#"
    ],
    "nounphrase": [
        "the #noun#",
        "the #adj# #noun#",
        "the #noun# #prepphrase#",
        "the #noun# and the #noun#",
        "#noun.a#",
        "#adj.a# #noun#",
        "the #noun# that #predicate#"
    ],
    "subject": subjects,
    "verb": past_tense_verbs,
    "noun": nouns,
    "adj": adjectives,
    "prepphrase": prep_phrases
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

'I Sued After Domestic Violence Call.'

In [47]:
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(12)])
print(fill(output, 60))

In L.A., Sample Made. Beyoncé Wrecked a Netflix. Jon Stewart
Got. For Gastric Sleeve, Britney Spears My Book Got a
Beyhive. Beyoncé Got. Britney Spears My Book Killed to L.A..
Billy Porter SCOTUS Got. You Dragged the Buster. On, True
Bathroom Humor ', " Michael Savarino Coach K's Grandson
Wrecked. JAYDAYOUNGAN Sued. WITH DOMESTIC VIOLENCE, Jennifer
Lopez Picked. WITH DOMESTIC VIOLENCE, What Got.
